In [5]:
import pandas as pd

# Load your Excel file
df = pd.read_excel("C:/Users/shiva/SupplyChainEmissionFactorsforUSIndustriesCommodities.xlsx")

# Display the first 5 rows
df.head()


,Name,Supply Chain GHG Emission Factors for US Commodities and Industries
0,Authors,"Wesley Ingwersen, Mo Li"
1,Description,Tables presenting supply chain and margin emis...
2,DOI,http://doi.org/10.23719/1517769
